In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Set up the display

In [2]:
def displayify_df(df):
    """For pretty displaying DataFrame in a notebook."""
    display_df = df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        }
    )
    display(display_df)

Load the data from the article

In [3]:
# load the documents
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("data/Trump/").load_data()

In [4]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

Generate rag dataset based on the data(from documents for this case)

In [5]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms.openai import OpenAI
import nest_asyncio

nest_asyncio.apply()

documents = documents  # a set of documents loaded by using for example a Reader

llm = OpenAI(model="gpt-4")

dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    llm=llm,
    num_questions_per_chunk=10,  # set the number of questions per nodes
)

rag_dataset = dataset_generator.generate_dataset_from_nodes()

take a look at the rag_dataset

In [6]:
rag_dataset.to_pandas()[:5]

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Who was the target of the assassination attemp...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,Former President Donald Trump,ai (gpt-4),ai (gpt-4)
1,What was the location of the assassination att...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The assassination attempt on Former President ...,ai (gpt-4),ai (gpt-4)
2,How did the Secret Service respond during the ...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The Secret Service shielded former President T...,ai (gpt-4),ai (gpt-4)
3,What injury did Former President Trump sustain...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,Former President Trump sustained a gunshot wou...,ai (gpt-4),ai (gpt-4)
4,How did the Former President describe his expe...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The Former President described his experience ...,ai (gpt-4),ai (gpt-4)


With our RAG (i.e query_engine) defined, we can make predictions (i.e., generate responses to the query) with it over the rag_dataset

In [7]:
prediction_dataset = await rag_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=100, show_progress=True
)

Batch processing of predictions: 100%|██████████| 10/10 [00:03<00:00,  3.10it/s]


Evaluating Answer and Context Relevancy Separately
We first need to define our evaluators (i.e. AnswerRelevancyEvaluator & ContextRelevancyEvaluator):

In [8]:
# instantiate the gpt-4 judges
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    AnswerRelevancyEvaluator,
    ContextRelevancyEvaluator,
)

judges = {}

judges["answer_relevancy"] = AnswerRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-3.5-turbo"),
)

judges["context_relevancy"] = ContextRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-4"),
)

Now, we can use our evaluator to make evaluations by looping through all of the <example, prediction> pairs

In [9]:
eval_tasks = []
for example, prediction in zip(
    rag_dataset.examples, prediction_dataset.predictions
):
    eval_tasks.append(
        judges["answer_relevancy"].aevaluate(
            query=example.query,
            response=prediction.response,
            sleep_time_in_seconds=1.0,
        )
    )
    eval_tasks.append(
        judges["context_relevancy"].aevaluate(
            query=example.query,
            contexts=prediction.contexts,
            sleep_time_in_seconds=1.0,
        )
    )

In [11]:
from tqdm.asyncio import tqdm_asyncio

In [12]:
eval_results1 = await tqdm_asyncio.gather(*eval_tasks[:250])

100%|██████████| 220/220 [00:16<00:00, 13.38it/s]


In [13]:
eval_results2 = await tqdm_asyncio.gather(*eval_tasks[250:])

0it [00:00, ?it/s]


In [14]:
eval_results = eval_results1 + eval_results2

In [15]:
evals = {
    "answer_relevancy": eval_results[::2],
    "context_relevancy": eval_results[1::2],
}

Taking a look at the evaluation results
Here we use a utility function to convert the list of EvaluationResult objects into something more notebook friendly. This utility will provide two DataFrames, one deep one containing all of the evaluation results, and another one which aggregates via taking the mean of all the scores, per evaluation method.

In [16]:
from llama_index.core.evaluation.notebook_utils import get_eval_results_df
import pandas as pd

deep_dfs = {}
mean_dfs = {}
for metric in evals.keys():
    deep_df, mean_df = get_eval_results_df(
        names=["baseline"] * len(evals[metric]),
        results_arr=evals[metric],
        metric=metric,
    )
    deep_dfs[metric] = deep_df
    mean_dfs[metric] = mean_df

In [17]:
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.850000
mean_context_relevancy_score,0.843182


We can get a look at the raw distribution of the scores by invoking value_counts() on the deep_df.

In [18]:
deep_dfs["answer_relevancy"]["scores"].value_counts()

scores
1.0    90
0.0    13
0.5     7
Name: count, dtype: int64

In [22]:
deep_dfs["context_relevancy"]["scores"].value_counts()

scores
1.000    74
0.750    13
0.000     7
0.875     6
0.625     3
0.250     3
0.125     2
0.375     1
0.500     1
Name: count, dtype: int64

Getting a closer look is made possible by viewing the records of any of the deep_df's.

In [20]:
displayify_df(deep_dfs["context_relevancy"].head(2))

In [21]:
cond = deep_dfs["context_relevancy"]["scores"] < 1
displayify_df(deep_dfs["context_relevancy"][cond].head(5))